# Import List

In [3]:
import OpenDartReader
import FinanceDataReader as fdr
import datetime
import numpy as np
import pandas as pd

# API Key

In [4]:
api_key = 'ebe76fbe2b073384d56fbf6d80e6831615d68acf'

# Method

In [5]:
###########################################################################################################################

# OpenDartReader Part

###########################################################################################################################

def return_finance_state(code, year):
    """
    inputs
        code -> [String] stock code
        year -> [int] year for searching yyyy
    outputs
        df -> [DataFrame] finance state
    """
    dart = OpenDartReader(api_key)
    df = dart.finstate(code, year)

    return df

def return_stock_num(code, year):
    dart = OpenDartReader(api_key)
    small = dart.report(code, '소액주주', year)
    return int(small.stock_tot_co.values[0].replace(",",""))

def return_stock_info_present(code):
    before_year = datetime.datetime.now().year - 1
    df = fdr.DataReader(code, str(before_year))
    return df.iloc[-1]

def return_finance_info(code, year, target="per"):
    """
    inputs
        code -> [String] stock code
        year -> [Int] year for searching yyyy
        target -> [String] target to know ex) "per"
    outputs
        value -> [Float] target value
    """
    finance_state_df = return_finance_state(code, year)
    present_stock_info = return_stock_info_present(code)
    num_of_stock = return_stock_num(code, year)

    if target == "per":
        net_profit = finance_state_df[np.logical_and(finance_state_df.account_nm=="당기순이익" , finance_state_df.fs_nm=="연결재무제표")].thstrm_amount.values[0]
        eps = int(net_profit.replace(",","")) / num_of_stock
        per = present_stock_info.Close / eps
        return per

    return -1

###########################################################################################################################

# FinanceDataReader Part

###########################################################################################################################
def get_stock_code_list(market_code):
    return fdr.StockListing(market_code)


def find_top_market_cap(stocks_df, year, top=20):
    """
    inputs
        stocks_df => [DataFrame] stock code list for searching from "utils.get_stock_code_list()"
        year =>  [INT] year info. ex) 2021
        top => [INT] num of rank for view (Default = 20)

    outpus
        [LIST] => list of stock code
    """

    # Make date info for input
    start_date = str(year) + "-01-01"
    end_date = str(year) + "-12-30"
    
    outs = []

    for raw_idx in range(stocks_df.shape[0]):
        code = stocks_df.iloc[raw_idx].Symbol
        name = stocks_df.iloc[raw_idx].Name
        try:
            df = fdr.DataReader(code, start_date, end_date)
            
        except:
            continue
        if 'Close' not in df.columns:
            continue
        elif 'Volume' not in df.columns:
            continue
        close_arr = df.Close.values
        volume_arr = df.Volume.values

        market_cap_arr = close_arr * volume_arr
        market_cap_mean_of_year = np.nanmean(market_cap_arr)
        
        outs.append((name, market_cap_mean_of_year))
    outs.sort(key=lambda x : x[1])
    print(outs[-top:])
    return outs[-top:]


# Report

In [6]:
code_list = get_stock_code_list('KOSPI')

In [7]:
print(code_list.shape)
code_list

(5378, 10)


,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,152100,KOSPI,ARIRANG 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7035,215620,KOSPI,흥국 S&P코리아로우볼,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7037,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시
7038,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7039,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [8]:
top_100_2018 = find_top_market_cap(stocks_df=code_list, year=2018, top=100)
top_100_2019 = find_top_market_cap(stocks_df=code_list, year=2019, top=100)
top_100_2020 = find_top_market_cap(stocks_df=code_list, year=2020, top=100)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[('한국금융지주', 16547753211.885246), ('하나제약', 23308306989.67213), ('한국항공우주', 34347826077.86885), ('하나금융지주', 42521858717.62295), ('하이브', nan), ('한화투자증권우', 103012294.26229508), ('한화솔루션우', 290219125.27459013), ('현대차3우B', 324209109.8360656), ('호전실업', 380403198.3606557), ('해태제과식품', 406498563.23770493), ('핸즈코퍼레이션', 426255601.02459013), ('현대퓨처넷', 494846886.2090164), ('호텔신라우', 542617696.1065574), ('현대차증권', 560471457.4180328), ('화성산업', 723916726.0245901), ('한화우', 765584908.1967213), ('한화3우B', 962270934.6311475), ('현대비앤지스틸우', 1057106660.4508197), ('형지엘리트', 1264677600.2745903), ('한화손해보험', 1581652951.352459), ('해성디에스', 1584213436.680328), ('한화투자증권', 1896098191.0450819), ('현대비앤지스틸', 1950871613.5655737), ('현대리바트', 2005591788.5245903), ('현대일렉트릭', 2973551296.7008195), ('현대홈쇼핑', 3448511110.655738), ('현대코퍼레이션홀딩스', 3675234562.295082), ('현대중공업지주', 4247949709.836066), ('현대차우', 4275472252.0491805), ('한화생명', 6515790348.155738), ('현대차2우B', 7448309161.885246), ('현대위아', 8028890730.737705), ('현대백화점', 10046556185.245